In [43]:
import pandas as pd
from loguru import logger
import json
import numpy as np
import plotly.express as px

%reload_ext autoreload
%autoreload 2

import sys
sys.path.append('..')
from src.plots import show_clients_dc, show_clients_per_dc, show_clients_per_dc_and_both, template
from src.data_for_tool import create_lists_days_weeks_months
from src.client_analysis import get_df_results_direct
import plotly.io as pio
pio.templates.default = template

In [44]:
# Import data
project_path = 'results'
dataset_name = 'df_limited'
df = pd.read_csv(f'{project_path}/{dataset_name}.csv', parse_dates=['Delivery date'])
dataset_name = 'df_distance_matrix'
df_distance_matrix = pd.read_csv(f'{project_path}/{dataset_name}.csv', index_col = 0)

# Import dict terminals
f = open("../data/external/intermodal_terminals/dict_terminals.json")
dict_terminals = json.load(f)

# Import dict points
f = open("../data/processed/dict_points.json")
dict_points = json.load(f)

#df_dc2, df_dc3, df_distance_matrix, dict_terminals, df_terminal = data_for_tool()
list_days, list_mondays, list_bi_daily, list_months = create_lists_days_weeks_months(2022)

df_dc1 = df[df["Shipper name"]=="DC1"]
df_dc2 = df[df["Shipper name"]=="DC2"]

len(df["Receiver name"].unique())

787

In [45]:
df.shape[0]

171859

### Distance analysis

In [39]:
df_dc1 = df_dc1.merge(df_distance_matrix["DC1"].reset_index().rename({"index": "Receiver name", "DC1": "distance dc-c"}, axis = 1), on = ["Receiver name"], how = "left")
df_dc2 = df_dc2.merge(df_distance_matrix["DC2"].reset_index().rename({"index": "Receiver name", "DC2": "distance dc-c"}, axis = 1), on = ["Receiver name"], how = "left")

In [36]:
def histogram_weight_distance(df_dc2, path_plot = False):
    df_dc2["distance dc-c (km)"] = df_dc2["distance dc-c"]/1000
    df_dc2["shipping weight (t)"] = df_dc2["Sender weight (kg)"]/1000
    fig = px.histogram(df_dc2, x="distance dc-c (km)", y="shipping weight (t)", 
                       marginal = 'box', 
                       nbins=100)
    if path_plot == False:
        fig.show()
    else:
        fig.write_html(path_plot, auto_open = False)
    return df_dc2

In [42]:
df_dc1 = histogram_weight_distance(df_dc1, "output/plots/histogram_weight_distance_dc1.html")
df_dc2 = histogram_weight_distance(df_dc2, "output/plots/histogram_weight_distance_dc2.html")

### plot_per_planning_horizon

In [34]:
def plot_per_planning_horizon(df_plot, planning_horizon, path_plot = False):
    df_plot["Sender weight (t)"] = df_plot["Sender weight (kg)"]/1000
    fig = make_subplots(rows=1, cols=2, 
                        shared_yaxes=True, 
                        horizontal_spacing=0.02, column_widths=[0.9, 0.1])
    if planning_horizon == "daily":
        df_plot["Planning date"] = df_plot["Delivery date"].dt.date
        df_grouped = df_plot.groupby(["Planning date"]).sum(numeric_only = True)
        title_xaxes = "Date"
    elif planning_horizon == "weekly":
        df_plot["Planning date"] = df_plot["Delivery date"].dt.isocalendar().week
        df_grouped = df_plot.groupby(["Planning date"]).sum(numeric_only = True)
        title_xaxes = "Week number"
    else:
        df_plot["Planning date"] = np.where(
                                    (df_plot["Delivery date"].dt.weekday == 0)|
                                    (df_plot["Delivery date"].dt.weekday == 1), "M", 
                                    np.where(
                                        (df_plot["Delivery date"].dt.weekday == 2)|
                                        (df_plot["Delivery date"].dt.weekday == 3), "W", "F"))
        df_plot["Planning week"] = df_plot["Delivery date"].dt.isocalendar().week.astype(str)
        df_plot["Planning date"] = df_plot["Planning date"] + " W"+ df_plot["Planning week"]
        df_grouped = df_plot.groupby(["Planning date"]).sum(numeric_only = True)
        title_xaxes = "Days"
    fig.add_trace(go.Bar(x = df_grouped.index, y=df_grouped["Sender weight (t)"], 
                         hovertemplate='''<br>%{x}<br>Weight: %{y:,.0f} kg<br><extra></extra>'''),1, 1)
    fig.add_trace(go.Box(y=df_grouped[df_grouped["Sender weight (t)"]>0]["Sender weight (t)"],
                         customdata = df_grouped[df_grouped["Sender weight (t)"]>0].index,
                        hovertemplate='''<br>%{customdata}<br>Weight: %{y:,.0f} kg<br><extra></extra>''', name = "Distribution"), 1, 2)
    fig.update_yaxes(
        title = "sum of sender weight (t)", 
        col = 1, row = 1, 
        side="left")
    fig.update_xaxes(title = title_xaxes, col = 1, row = 1)
    if planning_horizon == "bi-daily":
        fig.update_xaxes(categoryorder='array', categoryarray= list(df_plot["Planning date"].unique()))
    fig.update_layout(showlegend = False, title_x = 0.5)
    if path_plot == False:
        fig.show()
    else:
        fig.write_html(path_plot, auto_open = False)

In [43]:
plot_per_planning_horizon(df_dc1, "daily", "output/plots/plot_per_planning_horizon_dc1_daily.html")
plot_per_planning_horizon(df_dc1, "bi-daily", "output/plots/plot_per_planning_horizon_dc1_bidaily.html")
plot_per_planning_horizon(df_dc1, "weekly", "output/plots/plot_per_planning_horizon_dc1_weekly.html")
plot_per_planning_horizon(df_dc2, "daily", "output/plots/plot_per_planning_horizon_dc2_daily.html")
plot_per_planning_horizon(df_dc2, "bi-daily", "output/plots/plot_per_planning_horizon_dc2_bidaily.html")
plot_per_planning_horizon(df_dc2, "weekly", "output/plots/plot_per_planning_horizon_dc2_weekly.html")

## Summary

In [44]:
show_clients_dc(df, "output/plots/show_clients_dc_final.html")
show_clients_dc(df_dc1, "output/plots/show_clients_dc1_final.html", "DC1")
show_clients_dc(df_dc2, "output/plots/show_clients_dc2_final.html", "DC2")
show_clients_per_dc(df, "output/plots/show_clients_per_dc_final.html")
show_clients_per_dc_and_both(df, "output/plots/show_clients_per_dc_and_both_final.html")

C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\plots.py:403: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\plots.py:403: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\plots.py:403: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

### Individual Client Analysis

#### Get datasets

In [149]:
df_results_direct = get_df_results_direct(df_dc1, df_dc2, df_distance_matrix, list_mondays, dict_terminals, "truck")

C:\Users\marle\AppData\Local\Temp\ipykernel_11896\3790790129.py:3: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\co2_modells.py:44: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

2023-09-04 19:58:05.722 | INFO     | src.co2_modells:co2_modell_direct:262 - {'DC1': [49.00551373, 1.179093518], 'C100': [48.73679297917374, 2.312579994235449]}
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\create_solution.py:77: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numer

(1, 8)
only truck:  133.00865155595957
(2, 12)
(2, 15)
railroad:  169.01218560458835
Total processing time: 0.0
(1, 8)
only truck:  1080.3652507481943
(1, 12)
(1, 15)
railroad:  469.17990883597156
Total processing time: 0.015625
(1, 8)
only truck:  322.75563961428026
(1, 12)
(1, 15)
railroad:  476.6223661866204
Total processing time: 0.0
(1, 8)
only truck:  439.1040673315402
(1, 12)
(1, 15)


2023-09-04 19:58:05.908 | INFO     | src.co2_modells:co2_modell_direct:271 - T9
2023-09-04 19:58:05.915 | INFO     | src.co2:mesoscopic_train:204 - 1
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\co2_modells.py:44: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

2023-09-04 19:58:05.935 | INFO     | src.co2_modells:co2_modell_direct:262 - {'DC1': [49.00551373, 1.179093518], 'C1025': [43.6383919947028, 5.077291999513875]}
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\create_solution.py:77: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.

2023-09-04 19:58:05.

railroad:  475.7818191814699
Total processing time: 0.015625
(1, 8)
only truck:  1029.0618775899493
(1, 12)
(1, 15)
railroad:  418.8160837849109
Total processing time: 0.015625
(1, 8)
only truck:  606.1767120386869
(1, 12)
(1, 15)
railroad:  335.5178991388535
Total processing time: 0.03125
(1, 8)
only truck:  88.25808901875
(2, 12)
(2, 15)
railroad:  281.303991077101
Total processing time: 0.03125
(1, 8)
only truck:  223.16986291833334
(2, 12)
(2, 15)


2023-09-04 19:58:06.124 | INFO     | src.co2_modells:co2_modell_direct:271 - T4
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\co2_modells.py:44: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

2023-09-04 19:58:06.136 | INFO     | src.co2_modells:co2_modell_direct:262 - {'DC1': [49.00551373, 1.179093518], 'C1030': [48.27576999006595, 2.7062730410631275]}
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\create_solution.py:77: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.

2023-09-04 19:58:06.176 | INFO     | src.co2_modells:co2_modell_direct:269 - [[], [], []

...

railroad:  535.3739054079161
Total processing time: 0.015625
(1, 8)
only truck:  416.19578535340906
(1, 12)
(1, 15)
railroad:  234.70393718416327
Total processing time: 0.015625
(1, 8)
only truck:  547.5327241264646
(1, 12)
(2, 15)
railroad:  562.4463748216338
Total processing time: 0.015625
(1, 8)
only truck:  777.3097725899369
(1, 12)
(2, 15)
railroad:  790.4830463538573
Total processing time: 0.015625
(1, 8)
only truck:  904.6634320391287
(1, 12)


C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\create_solution.py:77: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.

2023-09-04 19:59:45.062 | INFO     | src.co2_modells:co2_modell_direct:269 - [[], [], [], [], ['C98'], [], [], [], [], []]
2023-09-04 19:59:45.062 | INFO     | src.co2_modells:co2_modell_direct:271 - T5
2023-09-04 19:59:45.078 | INFO     | src.co2:mesoscopic_train:204 - 1
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\co2_modells.py:44: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

2023-09-04 19:59:45.093 | INFO     | src.co2_modells:co2_m

(1, 15)
railroad:  342.7063939034368
Total processing time: 0.03125
(1, 8)
only truck:  776.5017359270959
(1, 12)
(1, 15)
railroad:  468.9290549312751
Total processing time: 0.015625
(1, 8)
only truck:  153.12863800738634
(2, 12)
(2, 15)
railroad:  161.37794626045203
Total processing time: 0.03125
(1, 8)
only truck:  852.2232436071843
(1, 12)
(1, 15)


2023-09-04 19:59:45.267 | INFO     | src.co2_modells:co2_modell_direct:269 - [[], [], [], [], ['C998'], [], [], [], [], []]
2023-09-04 19:59:45.267 | INFO     | src.co2_modells:co2_modell_direct:271 - T5
2023-09-04 19:59:45.282 | INFO     | src.co2:mesoscopic_train:204 - 1


railroad:  391.15748615917164
Total processing time: 0.03125
1


In [21]:
df_results_direct_train = get_df_results_direct(df_dc1, df_dc2, df_distance_matrix, list_mondays, dict_terminals, "train")

C:\Users\marle\AppData\Local\Temp\ipykernel_11896\3790790129.py:3: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\co2_modells.py:44: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

2023-09-04 18:10:12.431 | INFO     | src.co2_modells:co2_modell_direct:262 - {'DC1': [49.00551373, 1.179093518], 'C100': [48.73679297917374, 2.312579994235449]}
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\create_solution.py:77: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numer

(60, 9)
only truck:  13088.220488461347
(2, 12)
(2, 16)


2023-09-04 18:10:12.525 | INFO     | src.co2_modells:co2_modell_direct:269 - [[], [], [], ['C100'], [], [], [], [], [], []]
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\co2_modells.py:44: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

2023-09-04 18:10:12.698 | INFO     | src.co2_modells:co2_modell_direct:262 - {'DC1': [49.00551373, 1.179093518], 'C1017': [43.38407442675264, 5.208858763110696]}
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\create_solution.py:77: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.

2023-09-04 18:10:12.761 | INFO     | src.co2_

railroad:  11632.642943383893
Total processing time: 0.140625
(60, 9)
only truck:  106646.26462900572
(1, 12)
(1, 16)


2023-09-04 18:10:12.840 | INFO     | src.co2:mesoscopic_train:204 - 20
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\co2_modells.py:44: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

2023-09-04 18:10:12.966 | INFO     | src.co2_modells:co2_modell_direct:262 - {'DC1': [49.00551373, 1.179093518], 'C102': [47.51384698670071, -0.4694020005266281]}
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\create_solution.py:77: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



...

(1, 12)
(1, 16)
railroad:  23179.772829754304
Total processing time: 0.09375
(60, 9)


2023-09-04 18:14:37.117 | INFO     | src.co2_modells:co2_modell_direct:262 - {'DC2': [45.65261999, 5.122074999], 'C994': [44.89616919388058, 4.881370904301186]}
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\create_solution.py:77: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.

2023-09-04 18:14:37.164 | INFO     | src.co2_modells:co2_modell_direct:269 - [[], [], [], [], [], ['C994'], [], [], [], []]


only truck:  13844.9742577536
(2, 12)
(2, 16)
railroad:  14726.014740200391
Total processing time: 0.109375
(60, 9)
only truck:  84346.61613197936


C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\co2_modells.py:44: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

2023-09-04 18:14:37.320 | INFO     | src.co2_modells:co2_modell_direct:262 - {'DC2': [45.65261999, 5.122074999], 'C998': [47.398856998141326, -1.1856290051646567]}
C:\Users\marle\OneDrive - tbs-education\00_Masterarbeit\Code_New\notebooks\..\src\create_solution.py:77: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.

2023-09-04 18:14:37.368 | INFO     | src.co2_modells:co2_modell_direct:269 - [[], [], [], [], ['C998'], [], [], [], [], []]
2023-09-04 18:14:37.417 | INFO     | src.co

(1, 12)
(1, 16)
railroad:  15417.156957043882
Total processing time: 0.09375


***

#### Analysis

In [12]:
def get_distribution(dc, dct, ftl_type,speed_train, speed_truck):
    df_results_direct = pd.read_csv(f"results/direct/df_results_direct_ftl_{ftl_type}_speed_{speed_train}_{speed_truck}.csv")
    logger.info(df_results_direct.shape[0])
    df_results_direct = add_recommendation(df_results_direct)
    #df_results_direct = df_results_direct[(df_results_direct["dc"]==dc)]
    df_results_direct = df_results_direct[(df_results_direct["dc"]==dc)&(df_results_direct["terminal allocation"]!=dct)]
    print(df_results_direct[(df_results_direct["dc"]==dc)&(df_results_direct["terminal allocation"]!=dct)]["Recommendation distance"].value_counts())
    print(df_results_direct[(df_results_direct["dc"]==dc)&(df_results_direct["terminal allocation"]!=dct)]["Recommendation time"].value_counts())
    print(df_results_direct[(df_results_direct["dc"]==dc)&(df_results_direct["terminal allocation"]!=dct)]["Recommendation"].value_counts())
    return df_results_direct

def show_average(name):
    print("road:",df_results_direct[f"{name} road"].mean())
    print("rail:",df_results_direct[f"{name} railroad"].mean())
    print("prehaul:",df_results_direct[f"{name} prehaul"].mean())
    print("mainhaul:",df_results_direct[f"{name} mainhaul"].mean())
    print("endhaul:",df_results_direct[f"{name} endhaul"].mean())

In [18]:
df_results_direct = pd.read_csv("results/direct/df_results_direct_ftl_truck_speed_80_80.csv", index_col = 0)

In [56]:
show_average("co2")

road: 563.8032139809975
rail: 401.73892213394777
prehaul: 80.61555391932757
mainhaul: 105.58944647673064
endhaul: 215.53392173788956


In [58]:
show_average("time")

road: 5.573234951790633
rail: 7.3630749494949495
prehaul: 0.7968904958677686
mainhaul: 4.018949609733701
endhaul: 2.1305681772268135


In [49]:
show_average("distance")

road: 510.6795569620253
rail: 649.9435223628693
prehaul: 64.46284669479607
mainhaul: 492.44850914205347
endhaul: 93.0321665260197


In [24]:
def correlation_matrix(df_original, list_variables):
    df = df_original.copy()
    column_names = {
        'co2 road': "GHG road (total)",
        'co2 railroad': "GHG railroad (total)", 
        'co2 mainhaul': "GHG railroad (mainhaul)",
        'co2 prehaul': "GHG railroad (prehaul)",
        'co2 endhaul': "GHG railroad (endhaul)",
        'distance road': "Distance DC-Client (road)",
        'haversine distance': "Distance DC-C (crow flies)",
        'distance railroad': "Distance DC-C (railroad)",
        'distance mainhaul': "Distance T-T (mainhaul)",
        'distance prehaul': "Distance DC-T (prehaul)",
        'distance endhaul': "Distance T-Client (endhaul)"
    }
    df["Recommendation"] = np.where(df["Recommendation"] == "Rail", 1, 0)
    df = df[df["terminal allocation"]!="T4"]
    df["GHG Savings Ratio"] = df["co2 railroad"]/df["co2 road"]
    df["GHG Savings Ratio"] = (1-df["GHG Savings Ratio"])*100
    df = df.rename(column_names, axis = 1)
    df = df[list_variables]
    df_corr = df.corr().loc[["GHG Savings Ratio"]]
    #df_used.loc["GHG Savings Ratio"]
    fig = go.Figure()
    fig.add_trace(
        go.Heatmap(
            x = df_corr.columns,
            y = df_corr.index,
            z = np.array(df_corr),
            text=df_corr.values,
            texttemplate='%{text:.2f}',
            colorscale=px.colors.diverging.RdBu,
            zmin=-1,
            zmax=1)
    )
    fig.update_layout(height = 300)
    fig.show()
    return df_corr

In [25]:
df_results_direct_ftl_truck = pd.read_csv("results/direct/df_results_direct_ftl_truck_speed_80_80.csv", index_col = 0)
df_results_direct_ftl_train = pd.read_csv("results/direct/df_results_direct_ftl_train_speed_80_80.csv", index_col = 0)

df_results_direct_ftl_train["Sender weight (t)"] = 1260
df_results_direct_ftl_truck["Sender weight (t)"] = 21

df_results_direct_all = pd.concat([df_results_direct_ftl_truck, df_results_direct_ftl_train]).drop("distance endhaul", axis = 1).merge(df_results_direct[["client","distance endhaul"]], on = "client", how = "left")
df_results_direct_all = df_results_direct_all[df_results_direct_all["dc"]=="DC1"]
df_results_direct_all["distance prehaul"] = 106308

list_variables = [
    "Sender weight (t)",
    "Distance DC-Client (road)",
    "Distance DC-T (prehaul)",
    "Distance T-T (mainhaul)", 
    "Distance T-Client (endhaul)", 
    "GHG Savings Ratio"]
df_used = correlation_matrix(df_results_direct_all[df_results_direct_all["dc"]=="DC1"], list_variables)

In [28]:
def plot_recommendation(df1, path_plot = False):
    night_colors = ['rgb(87, 179, 142)', 'rgb(248, 114, 104)']
    fig = go.Figure()
    fig.add_trace(go.Pie(values=df1["Recommendation"].value_counts().values, 
                         labels=df1["Recommendation"].value_counts().index, hole = 0.3, opacity = 0.7, marker_colors = night_colors, textinfo = "label+percent", textfont = {"size":20, "family": "arial", "color": "black"}))
    fig.update_layout(width = 500, showlegend = False)
    if path_plot is False:
        fig.show()
    else:
        fig.write_html(path_plot, auto_open = False)

plot_recommendation(df_results_direct_ftl_truck)
plot_recommendation(df_results_direct_ftl_train)

In [32]:
def boxplot_distribution(df, path_plot = False):
    fig = go.Figure()
    night_colors = ['rgb(87, 179, 142)', 'rgb(248, 114, 104)']
    fig.add_trace(go.Box(x = df[df["Recommendation"]=="Rail"]["haversine distance"], name = "Rail",  marker_color = night_colors[0]))
    fig.add_trace(go.Box(x = df[df["Recommendation"]=="Road"]["haversine distance"], name = "Road",  marker_color = night_colors[1]))
    fig.update_traces(boxpoints = False)
    fig.update_layout(xaxis_title='Distance DC-Client [km]')
    if path_plot == False:
        fig.show()
    else:
        fig.write_html(path_plot, auto_open = False)

boxplot_distribution(df_results_direct_all)